In [ ]:
!pip install datasets faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 8.7 MB/s eta 0:00:00


In [ ]:
import datasets
import faiss
print("Required libraries are installed correctly.")

Required libraries are installed correctly.


In [ ]:
pip install kagglehub torch transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 57.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 41.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 35.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 70.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
# -*- coding: utf-8 -*-
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import kagglehub
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from transformers import RagTokenizer, RagRetriever, RagSequenceForGeneration
from sklearn.model_selection import train_test_split
from glob import glob

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("kmader/skin-cancer-mnist-ham10000")

print("Path to dataset files:", path)

In [ ]:
base_skin_dir = os.path.join('..', 'input')

# Merging images from both folders HAM10000_images_part1.zip and HAM10000_images_part2.zip into one dictionary
imageid_path_dict = {os.path.splitext(os.path.basename(x))[0]: x
                     for x in glob(os.path.join(base_skin_dir, '*', '*.jpg'))}

# This dictionary is useful for displaying more human-friendly labels later on
lesion_type_dict = {
    'nv': 'Melanocytic nevi',
    'mel': 'Melanoma',
    'bkl': 'Benign keratosis-like lesions ',
    'bcc': 'Basal cell carcinoma',
    'akiec': 'Actinic keratoses',
    'vasc': 'Vascular lesions',
    'df': 'Dermatofibroma'
}

In [ ]:
# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# Downloading HAM10000 dataset
dataset_path = kagglehub.dataset_download("kmader/skin-cancer-mnist-ham10000")
base_skin_dir = os.path.join('..', 'input')

# Mapping lesion types to readable names
lesion_type_dict = {
    'nv': 'Melanocytic nevi',
    'mel': 'Melanoma',
    'bkl': 'Benign keratosis-like lesions',
    'bcc': 'Basal cell carcinoma',
    'akiec': 'Actinic keratoses',
    'vasc': 'Vascular lesions',
    'df': 'Dermatofibroma',
    'unknown': "Random"
}

# metadata
metadata = pd.read_csv(os.path.join(dataset_path, "HAM10000_metadata.csv"))

# Mapping image IDs to file paths
image_paths = {os.path.splitext(os.path.basename(x))[0]: x for x in glob(os.path.join(dataset_path, "HAM10000_images_part_1", "*.jpg"))}
image_paths.update({os.path.splitext(os.path.basename(x))[0]: x for x in glob(os.path.join(dataset_path, "HAM10000_images_part_2", "*.jpg"))})

# Add file paths and lesion type information to metadata
metadata['path'] = metadata['image_id'].map(image_paths.get)
metadata['cell_type'] = metadata['dx'].map(lesion_type_dict.get)
metadata['cell_type_idx'] = pd.Categorical(metadata['cell_type']).codes

# 2. Preprocess Data
# ------------------------------------------------------------------------------
# Defining diagnosis to numerical labels mapping
diag_dict = {
    'akiec': 0, 'bcc': 1, 'bkl': 2, 'df': 3, 'mel': 4, 'nv': 5, 'vasc': 6
}

# Map diagnosis to numerical labels
metadata['label'] = metadata['dx'].map(diag_dict)

# Check the number of samples in the dataset
print(f"Samples in dataset: {len(metadata)}")

In [ ]:
# Now lets see the sample of tile_df to look on newly made columns
metadata.head()

In [ ]:
metadata.isnull().sum()

In [ ]:
metadata['age'].fillna((metadata['age'].mean()), inplace=True)

In [ ]:
metadata.isnull().sum()

In [ ]:
# Load and preprocess images
def load_and_preprocess(image_path, label):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [224, 224])
    image = tf.keras.applications.resnet50.preprocess_input(image)
    return image, label

# Filter out rows with missing image paths
metadata = metadata.dropna(subset=['path'])

# Create TensorFlow Dataset
dataset = tf.data.Dataset.from_tensor_slices(
    (metadata['path'].values, metadata['label'].values))
dataset = dataset.map(load_and_preprocess)

# Shuffle the dataset (optional, but recommended)
dataset = dataset.shuffle(1000)

# Split dataset into first 100 for training and next 50 for testing
train_dataset = dataset.take(500).batch(32).prefetch(buffer_size=tf.data.AUTOTUNE)
test_dataset = dataset.skip(500).take(90).batch(32).prefetch(buffer_size=tf.data.AUTOTUNE)

# Count the number of samples in the training and test datasets
train_samples = sum(1 for _ in train_dataset.unbatch())
test_samples = sum(1 for _ in test_dataset.unbatch())

print(f"Samples in training dataset: {train_samples}")
print(f"Samples in test dataset: {test_samples}")

**Model trainig and fitting**

In [ ]:
# Install required libraries
!pip install -q transformers faiss-cpu wikipedia Pillow tensorflow

from io import BytesIO
import numpy as np
from PIL import Image
import ipywidgets as widgets
from IPython.display import display
import faiss
import torch
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from transformers import CLIPProcessor, CLIPModel, AutoTokenizer, AutoModelForSeq2SeqLM
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model

# Define device based on GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Initialize CLIP components
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)

# 1. Load and Customize ResNet50 Model ---------------------------------------
print("🚀 Loading and customizing ResNet50 model...")
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add custom head
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(7, activation='softmax')(x)  # 7 classes for skin lesions

# Create final model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze base layers
for layer in base_model.layers:
    layer.trainable = False


checkpoint_filepath = "model700.h5"
model_checkpoint = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_best_only=True,
    monitor='val_accuracy',  # Track validation accuracy
    mode='max',  # Save the model with the highest validation accuracy
    verbose=1
)

new_model = tf.keras.models.load_model(checkpoint_filepath)
new_model.summary()
# Compile model
# model.compile(optimizer=tf.keras.optimizers.Adam(1e-4),
#               loss='sparse_categorical_crossentropy',
#               metrics=['accuracy'])
# model.fit(train_dataset, validation_data=test_dataset, epochs=10)
# model.summary()
# model.fit(
#     train_generator,
#     steps_per_epoch=train_generator.samples // train_generator.batch_size,  # Number of batches per epoch
#     validation_data=validation_generator,
#     validation_steps=validation_generator.samples // validation_generator.batch_size,
#     epochs=10,  # Number of epochs
#     verbose=1
# )
# Verify architecture

Using device: cpu
🚀 Loading and customizing ResNet50 model...


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2             │ (None, 224, 224, 3)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_pad (ZeroPadding2D) │ (None, 230, 230, 3)    │              0 │ input_layer_2[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_conv (Conv2D)       │ (None, 112, 112, 64)   │          9,472 │ conv1_pad[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_bn                  │ (None, 112, 112, 64)   │            256 │ conv1_conv[0][0]       │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_relu (Activation)   │ (None, 112, 112, 64)   │              0 │ conv1_bn[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ pool1_pad (ZeroPadding2D) │ (None, 114, 114, 64)   │              0 │ conv1_relu[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ pool1_pool (MaxPooling2D) │ (None, 56, 56, 64)     │              0 │ pool1_pad[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_conv       │ (None, 56, 56, 64)     │          4,160 │ pool1_pool[0][0]       │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_bn         │ (None, 56, 56, 64)     │            256 │ conv2_block1_1_conv[0… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_relu       │ (None, 56, 56, 64)     │              0 │ conv2_block1_1_bn[0][… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_conv       │ (None, 56, 56, 64)     │         36,928 │ conv2_block1_1_relu[0… │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_bn         │ (None, 56, 56, 64)     │            256 │ conv2_block1_2_conv[0… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_relu       │ (None, 56, 56, 64)     │              0 │ conv2_block1_2_bn[0][… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_0_conv       │ (None, 56, 56, 256)    │         16,640 │ pool1_pool[0][0]       │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_3_conv       │ (None, 56, 56, 256)    │         16,640 │ conv2_block1_2_relu[0… │
│ (Conv2D)             

 Total params: 25,693,065 (98.01 MB)

 Trainable params: 2,105,351 (8.03 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

 Optimizer params: 2 (12.00 B)

In [ ]:

# 2. ResNet50-Based Classifier with Top-k Predictions ------------------------
class ResNetClassifier:
    def __init__(self, model):
        self.model = model
        self.class_names = {
            0: 'Melanoma',
            1: 'Actinic keratosis',
            2: 'Basal cell carcinoma',
            3: 'Benign keratosis',
            4: 'Dermatofibroma',
            5: 'Vascular lesion',
            6: 'Squamous cell carcinoma'
        }

    def preprocess_image(self, image):
        """Preprocess image for ResNet50."""
        img = image.resize((224, 224))
        img_array = tf.keras.preprocessing.image.img_to_array(img)
        return tf.expand_dims(
            tf.keras.applications.resnet50.preprocess_input(img_array),
            0
        )

    def predict(self, image, top_k=3):
        """Predict top-k skin lesion classes."""
        logits = self.model.predict(self.preprocess_image(image))
        probabilities = tf.nn.softmax(logits).numpy().flatten()
        top_k_indices = np.argsort(probabilities)[-top_k:][::-1]
        return [
            {
                'class_id': int(idx),
                'class_name': self.class_names[idx],
                'confidence': float(probabilities[idx])
            }
            for idx in top_k_indices
        ]

# 3. Differential Analyzer Approach (DAA) ------------------------------------
class DifferentialAnalyzer:
    def __init__(self, threshold=0.5):
        self.threshold = threshold

    def select_features(self, features):
        """
        Select features using the Differential Analyzer Approach (DAA).
        :param features: List of feature values (sorted in descending order).
        :return: Selected features based on the slope threshold.
        """
        selected_features = []
        for i in range(1, len(features)):
            slope = (features[i] - features[i - 1]) / (i - (i - 1))  # Slope calculation
            if slope >= self.threshold:
                selected_features.append(features[i - 1])
        return selected_features

# 4. Medical Retriever --------------------------------------------------------
class MedicalRetriever:
    def __init__(self):
        self.index = faiss.IndexFlatIP(512)  # Cosine similarity
        self.documents = []
        self._initialize_knowledge_base()

    def _initialize_knowledge_base(self):
        medical_texts = [
              "Melanoma: ABCD criteria - Asymmetry, Border irregularity, Color variation, Diameter greater than 6mm.",
              "Actinic keratosis treatment: Cryotherapy or 5-fluorouracil for precancerous lesions.",
              "Basal cell carcinoma: Pearly appearance with telangiectasia, often on sun-exposed areas.",
              "Benign keratosis: Waxy, stuck-on appearance, typically non-cancerous.",
              "Dermatofibroma: Firm papule with dimple sign, usually harmless.",
              "Vascular lesions: Red or purple coloration, often benign.",
              "Squamous cell carcinoma: Scaly red patches with irregular borders, potential for metastasis.",
              "Merkel cell carcinoma: Rare, aggressive cancer presenting as painless, firm, red or purple nodule.",
              "Kaposi’s sarcoma: Purple, red, or brown patches or nodules, often associated with immunocompromised states.",
              "Sebaceous gland carcinoma: Rare, aggressive tumor arising from sebaceous glands, often on eyelids.",
              "Porocarcinoma: Malignant tumor of sweat glands, presenting as firm, dome-shaped nodule.",
              "Atypical fibroxanthoma: Rare, low-grade sarcoma presenting as rapidly growing nodule.",
              "Keratoacanthoma: Rapidly growing, dome-shaped nodule with central keratin plug.",
              "Dermatofibrosarcoma protuberans: Slow-growing, firm nodule with a characteristic 'stuck-on' appearance.",
              "Angiosarcoma: Rare, aggressive cancer of blood vessels, presenting as bruise-like lesions.",
              "Lymphoma of the skin: Presents as red, scaly patches or plaques, often misdiagnosed as eczema.",
              "Mycosis fungoides: A type of cutaneous T-cell lymphoma presenting as scaly patches or plaques.",
              "Extramammary Paget disease: Red, scaly, itchy patches in genital or perianal areas.",
              "Basal cell nevus syndrome: Genetic condition leading to multiple basal cell carcinomas.",
              "Xeroderma pigmentosum: Genetic disorder causing extreme sensitivity to UV light, leading to early skin cancers.",
              "Albinism: Genetic condition causing lack of melanin, increasing risk of skin cancer.",
              "Erythropoietic protoporphyria: Rare genetic disorder causing skin sensitivity to sunlight.",
              "Porphyria cutanea tarda: Disorder causing skin fragility and blistering, often triggered by sun exposure.",
              "Alkaptonuria: Rare metabolic disorder leading to ochronosis and increased risk of skin cancer.",
              "Nevoid basal cell carcinoma syndrome: Genetic condition leading to multiple basal cell carcinomas.",
              "Gardner's syndrome: Genetic disorder causing multiple polyps in the colon and multiple skin tumors.",
              "Pachyonychia congenita: Rare genetic disorder causing thickened nails and skin lesions.",
              "Ehlers-Danlos syndrome: Connective tissue disorder causing hyperelastic skin and increased risk of skin cancer.",
              "Marfan syndrome: Connective tissue disorder causing tall stature and increased risk of skin cancer.",
              "Neurofibromatosis type 1: Genetic disorder causing multiple benign tumors and increased risk of skin cancer.",
              "Tuberous sclerosis: Genetic disorder causing benign tumors in multiple organs, including skin.",
              "Ataxia-telangiectasia: Genetic disorder causing progressive neurological problems and increased risk of skin cancer.",
              "Bloom syndrome: Rare genetic disorder causing short stature and increased risk of skin cancer.",
              "Fanconi anemia: Genetic disorder causing bone marrow failure and increased risk of skin cancer.",
              "Shwachman-Diamond syndrome: Rare genetic disorder causing pancreatic insufficiency and increased risk of skin cancer.",
              "Chediak-Higashi syndrome: Rare genetic disorder causing immune deficiency and increased risk of skin cancer.",
              "Wiskott-Aldrich syndrome: Genetic disorder causing immune deficiency and increased risk of skin cancer.",
              "Hyper-IgE syndrome: Rare genetic disorder causing immune deficiency and increased risk of skin cancer.",
              "Job syndrome: Rare genetic disorder causing immune deficiency and increased risk of skin cancer.",
              "Chronic granulomatous disease: Genetic disorder causing immune deficiency and increased risk of skin cancer.",
              "Leukocyte adhesion deficiency: Rare genetic disorder causing immune deficiency and increased risk of skin cancer.",
              "Hyper-IgM syndrome: Genetic disorder causing immune deficiency and increased risk of skin cancer.",
              "Severe combined immunodeficiency: Genetic disorder causing severe immune deficiency and increased risk of skin cancer.",
              "DiGeorge syndrome: Genetic disorder causing immune deficiency and increased risk of skin cancer.",
              "X-linked lymphoproliferative syndrome: Genetic disorder causing immune deficiency and increased risk of skin cancer.",
              "Hyper-IgG syndrome: Rare genetic disorder causing immune deficiency and increased risk of skin cancer.",
              "Hyper-IgA syndrome: Rare genetic disorder causing immune deficiency and increased risk of skin cancer.",
              "Hyper-IgM syndrome: Rare genetic disorder causing immune deficiency and increased risk of skin cancer.",
              "Hyper-IgE syndrome: Rare genetic disorder causing immune deficiency and increased risk of skin cancer.",
              "Hyper-IgG syndrome: Rare genetic disorder causing immune deficiency and increased risk of skin cancer.",
              "Hyper-IgA syndrome: Rare genetic disorder causing immune deficiency and increased risk of skin cancer.",
              "Hyper-IgM syndrome: Rare genetic disorder causing immune deficiency and increased risk of skin cancer.",
              "Hyper-IgE syndrome: Rare genetic disorder causing immune deficiency and increased risk of skin cancer.",
              "Hyper-IgG syndrome: Rare genetic disorder causing immune deficiency and increased risk of skin cancer.",
              "Hyper-IgA syndrome: Rare genetic disorder causing immune deficiency and increased risk of skin cancer.",
              "Hyper-IgM syndrome: Rare genetic disorder causing immune deficiency and increased risk of skin cancer.",
              "Hyper-IgE syndrome: Rare genetic disorder causing immune deficiency and increased risk of skin cancer.",
              "Hyper-IgG syndrome: Rare genetic disorder causing immune deficiency and increased risk of skin cancer.",
              "Hyper-IgA syndrome: Rare genetic disorder causing immune deficiency and increased risk of skin cancer.",
              "Hyper-IgM syndrome: Rare genetic disorder causing immune deficiency and increased risk of skin cancer.",
              "Hyper-IgE syndrome: Rare genetic disorder causing immune deficiency and increased risk of skin cancer.",
              "Hyper-IgG syndrome: Rare genetic disorder causing immune deficiency and increased risk of skin cancer.",
              "Hyper-IgA syndrome: Rare genetic disorder causing immune deficiency and increased risk of skin cancer.",
              "Hyper-IgM syndrome: Rare genetic disorder causing immune deficiency and increased risk of skin cancer.",
              "Hyper-IgE syndrome: Rare genetic disorder causing immune deficiency and increased risk of skin cancer.",
              "Hyper-IgG syndrome: Rare genetic disorder causing immune deficiency and increased risk of skin cancer.",
              "Hyper-IgA syndrome: Rare genetic disorder causing immune deficiency and increased risk of skin cancer.",
              "Hyper-IgM syndrome: Rare genetic disorder causing immune deficiency and increased risk"]
        for text in medical_texts:
            inputs = clip_processor(text=text, return_tensors="pt", padding=True)
            inputs = {k: v.to(device) for k, v in inputs.items()}
            text_features = clip_model.get_text_features(**inputs).detach().cpu().numpy()
            faiss.normalize_L2(text_features)
            self.documents.append(text)
            self.index.add(text_features.astype('float32'))

    def retrieve(self, query_embedding, k=3):
        """Retrieve relevant medical information."""
        query_embedding = np.asarray(query_embedding, dtype='float32')
        if len(query_embedding.shape) == 1:
            query_embedding = np.expand_dims(query_embedding, 0)
        faiss.normalize_L2(query_embedding)
        return [self.documents[i] for i in self.index.search(query_embedding, k)[1][0]]

# 5. MedicalRAG with DAA-Deep and Flan-T5 Transformer -------------------------
class MedicalRAG:
    def __init__(self, classifier, retriever):
        self.classifier = classifier
        self.retriever = retriever
        self.daa = DifferentialAnalyzer(threshold=0.5)  # Initialize DAA

        # Use Flan-T5 for medical text generation
        self.tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-large")
        self.generator = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-large")
        self.generator.to(device)
        self.device = device
        self.report_template = """CLINICAL ANALYSIS REPORT:
        Visual Findings: {context}
        Differential Diagnoses:
        {differential_diagnoses}
        Supporting Evidence: {evidence}
        """

    def generate_report(self, image, top_k=3):
        if image is None:
            raise ValueError("No image provided for analysis.")

        try:
            # Process image with CLIP
            inputs = clip_processor(images=image, return_tensors="pt")
            inputs = {k: v.to(device) for k, v in inputs.items()}
            image_features = clip_model.get_image_features(**inputs)
            image_embedding = image_features.detach().cpu().numpy()

            # Retrieve context
            context_docs = self.retriever.retrieve(image_embedding)
            if not context_docs:
                raise ValueError("No relevant medical information found.")

            # Get top-k predictions
            diagnoses = self.classifier.predict(image, top_k=top_k)

            # Apply DAA for feature selection
            feature_values = [diagnosis['confidence'] for diagnosis in diagnoses]
            selected_features = self.daa.select_features(feature_values)

            # Filter diagnoses based on selected features
            filtered_diagnoses = [diagnoses[i] for i in range(len(diagnoses)) if feature_values[i] in selected_features]

            # Format differential diagnoses
            differential_diagnoses = "\n".join([
                f"- {diagnosis['class_name']} ({diagnosis['confidence']:.1%} confidence)"
                for diagnosis in filtered_diagnoses
            ])

            # Generate report
            report = self.report_template.format(
                context='\n- '.join(context_docs),
                differential_diagnoses=differential_diagnoses,
                evidence='\n'.join([f"• {doc.split(':')[1].strip()}" for doc in context_docs[:2]])
            )

            # Generate explanation using Flan-T5
            prompt = f"Explain the differential diagnosis for: {differential_diagnoses} using: {context_docs}"
            inputs = self.tokenizer(
                prompt,
                return_tensors="pt",
                max_length=512,
                truncation=True,
                padding='max_length'
            )
            inputs = {k: v.to(self.device) for k, v in inputs.items()}

            outputs = self.generator.generate(
                input_ids=inputs['input_ids'],
                attention_mask=inputs['attention_mask'],
                max_length=512,
                num_beams=4,
                early_stopping=True
            )

            explanation = self.tokenizer.decode(outputs[0], skip_special_tokens=True)

            return report + "\nEXPERT ANALYSIS:\n" + explanation

        except Exception as e:
            raise RuntimeError(f"Report generation failed: {str(e)}") from e
    def answer_question(self, question, report):
        """Generate an answer to a follow-up question using Flan-T5 with improved context handling."""
        if not report:
            raise ValueError("No report available for answering the question.")

        try:
            # Enhanced prompt template with better context structuring
            prompt = f"""
            Medical Context:
            {report}

            User Question: {question}

            Generate a answer:
            """

            # Enhanced generation parameters
            outputs = self.generator.generate(
                input_ids=self.tokenizer(prompt, return_tensors="pt", max_length=1024, truncation=True).input_ids.to(self.device),
                max_length=400,
                num_beams=5,
                temperature=0.7,
                do_sample=True,
                top_p=0.95,
                no_repeat_ngram_size=3,
                early_stopping=True
            )
            print(outputs)
            answer = self.tokenizer.decode(outputs[0], skip_special_tokens=True).strip()
            print(answer,"q23")
            # Format the answer with structured sections


            return answer.strip()

        except Exception as e:
            return f"⚠ Error in processing your question: {str(e)}. Please rephrase or consult a medical professional."

# 6. MedicalAIAssistant ------------------------------------------------------
class MedicalAIAssistant:
    def __init__(self):
        self.classifier = ResNetClassifier(model)
        self.retriever = MedicalRetriever()
        self.rag = MedicalRAG(self.classifier, self.retriever)
        self.current_case = None
        self._setup_ui()

    def _setup_ui(self):
        """Set up the user interface."""
        style = {'description_width': '100px'}
        layout = widgets.Layout(width='auto', margin='10px 0')

        self.uploader = widgets.FileUpload(
            description="Upload Lesion:",
            style=style,
            accept='image/*',
            layout=layout
        )

        self.disclaimer = widgets.HTML(
            value="<div style='color: #d32f2f; padding: 10px; border: 1px solid #ffcdd2; background-color: #ffebee;'>"
                  "<b>Important:</b> This AI provides preliminary analysis only. Always consult a dermatologist."
                  "</div>"
        )

        self.analyze_btn = widgets.Button(
            description="Analyze Image",
            icon='microscope',
            style={'button_color': '#4CAF50'},
            layout=layout
        )

        self.ask_box = widgets.Text(
            placeholder="Ask follow-up question...",
            disabled=True,
            layout=layout
        )

        self.ask_btn = widgets.Button(
            description="Ask Question",
            icon='comment-medical',
            style={'button_color': '#2196F3'},
            disabled=True,
            layout=layout
        )

        self.chat_history = widgets.Output(
            layout=widgets.Layout(width='90%', height='400px', overflow='auto')
        )

        self.ui = widgets.VBox([
            self.disclaimer,
            self.uploader,
            widgets.HBox([self.analyze_btn, self.ask_btn]),
            self.ask_box,
            self.chat_history
        ])

    def _clear_chat(self):
        """Clear the chat history."""
        self.chat_history.clear_output()

    def _handle_error(self, error_msg):
        """Handle errors gracefully."""
        with self.chat_history:
            print(f"⛔ Error: {error_msg}")
            print("Please try again or consult a medical professional.")

    def analyze_lesion(self):
        """Analyze a skin lesion and generate a differential diagnosis report."""
        display(self.ui)

        def on_analyze_click(_):
            self._clear_chat()
            self.ask_btn.disabled = True
            self.ask_box.disabled = True

            try:
                if not self.uploader.value:
                    raise ValueError("No image uploaded")

                img_data = list(self.uploader.value.values())[0]['content']
                image = Image.open(BytesIO(img_data))

                with self.chat_history:
                    print("🔍 Analyzing lesion...")
                    display(image.resize((224, 224)))

                    report = self.rag.generate_report(image, top_k=3)
                    self.current_case = report

                    self.ask_btn.disabled = False
                    self.ask_box.disabled = False

                    print("\n📝 Medical Report:")
                    print(report.split("EXPERT ANALYSIS:")[0])
                    print("\n💡 Ask follow-up questions about the diagnosis!")

            except Exception as e:
                self._handle_error(f"Analysis failed: {str(e)}")
                self.current_case = None

        def on_ask_click(_):
            """Handle follow-up questions."""
            question = self.ask_box.value.strip()
            if question and self.current_case:
                with self.chat_history:
                    print(f"\n👤 Question: {question}")
                    try:
                        answer = self.rag.answer_question(question, self.current_case)
                        print(f"🤖 AI Assistant: {answer}")
                    except Exception as e:
                        self._handle_error(f"Question processing failed: {str(e)}")
                self.ask_box.value = ""

        self.analyze_btn.on_click(on_analyze_click)
        self.ask_btn.on_click(on_ask_click)
        self.ask_box.on_submit(on_ask_click)

# Initialize and run the system
print("🚀 Initializing Medical AI Assistant with DAA-Deep...")
assistant = MedicalAIAssistant()
assistant.analyze_lesion()

🚀 Initializing Medical AI Assistant with DAA-Deep...


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]